<a href="https://colab.research.google.com/github/JC1201/Intelligent-Edge-Caching-Using-Federated-Learning-to-Predict-Content-Popularity/blob/main/TYP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy
from tensorflow.python.ops.gen_dataset_ops import shuffle_dataset
from sklearn.model_selection import train_test_split


dataset, info = tfds.load("movielens/100k-ratings", split=["train"], with_info=True)

data_set = dataset[0]

def preprocess_data(dataset):

    user_id = tf.strings.to_number(dataset["user_id"], out_type=tf.int32)
    movie_id = tf.strings.to_number(dataset["movie_id"], out_type=tf.int32)

    #casting user rating into 0 and 1 (0 means didn't rate this movie, 1 means did rate this movie)

    did_rate = tf.cast(dataset["user_rating"] > 0, tf.int64)

    features = {
        "user_id": user_id,
        "movie_id": movie_id,
        "user_rating":  did_rate
    }

    return features

data_set = data_set.map(preprocess_data)

print(len(data_set))

num_client = 5
userS_dataset = []

dataset_size = len(data_set)

for i in range(num_client):

    seed = numpy.random.seed(i)
    data_set = data_set.shuffle(buffer_size=dataset_size, seed= seed)
    userS_dataset.append(data_set)

data_set_1 =  data_set.shuffle(buffer_size=dataset_size, seed=42)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-ratings/incomplete.5S90RY_0.1.1/movielens-train.tfrecord*..…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.1. Subsequent calls will reuse this data.
100000


In [2]:
#size of training data and testing data
train_size = int(dataset_size * 0.8)
test_size = dataset_size - train_size

#extract training data and testing data
train_data_1 = data_set_1.take(int(train_size))
test_data_1 = data_set_1.skip(int(train_size))

train_data = []
test_data = []

for i in range(num_client):

    train_data.append(userS_dataset[i].take(int(train_size)))
    test_data.append(userS_dataset[i].skip(int(train_size)))


In [3]:
userIDs = []
movieIDs = []
ratings = []

unique_user_movie_pair = set()
number = 0
for example in data_set:

    # Convert TensorFlow tensor to a NumPy value
    user_id = example["user_id"].numpy()
    movie_id = example["movie_id"].numpy()
    rating = example["user_rating"].numpy()

    # Add movie pair only if it is uniuqe
    if (user_id, movie_id) not in unique_user_movie_pair:
        userIDs.append(user_id)
        movieIDs.append(movie_id)
        ratings.append(rating)
        unique_user_movie_pair.add((user_id, movie_id))
        number += 1

print(f"Number of unique movie pairs: {number}")

# Convert lists to NumPy arrays for later processing
userIDs = numpy.array(userIDs)
movieIDs = numpy.array(movieIDs)
ratings = numpy.array(ratings)

movieIDs = numpy.unique(movieIDs)
userIDs = numpy.unique(userIDs)

#size of user and movie in training data
num_user = max(userIDs)
num_movie = max(movieIDs)

print(f"Number of unique users: {num_user}")
print(f"Number of unique movies: {num_movie}")


Number of unique movie pairs: 100000
Number of unique users: 943
Number of unique movies: 1682


In [4]:
triplets = list(zip(userIDs, movieIDs, ratings))

train_triplets, test_triplets = train_test_split(triplets, test_size=0.2, random_state=42)
train_matrix = numpy.zeros((num_user, num_movie), dtype=numpy.int32)
test_matrix = numpy.zeros((num_user, num_movie), dtype=numpy.int32)

def integrate_feature_into_matrix(userIDs, movieIDs, ratings, num_user, num_movie, matrix):

    # Create a 2D matrix filled with zeros

    # Populate the matrix
    for userID, movieID, rating in zip(userIDs, movieIDs, ratings):
        matrix[int(userID), int(movieID)] = rating

    return matrix

for user_id, movie_id, rating in train_triplets:
    train_matrix[user_id-1, movie_id-1] = rating

for user_id, movie_id, rating in test_triplets:
    test_matrix[user_id-1, movie_id-1] = rating

print(train_matrix.shape)
print(test_matrix.shape)
train_matrix = train_matrix.reshape((1, num_user, num_movie, 1))

print(train_matrix.shape)
print(test_matrix.shape)

(943, 1682)
(943, 1682)
(1, 943, 1682, 1)
(943, 1682)


In [65]:
def build_cnn_model(num_user, num_movie):

    input_matrix = tf.keras.layers.Input(shape=(num_user, num_movie, 1), batch_size=64, name="user_id")

    # First Convolution Block
    cnn_layer = tf.keras.layers.Conv2D(128, kernel_size=(3, 3))(input_matrix)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(cnn_layer)
    cnn_layer = tf.keras.layers.Dropout(0.2)(cnn_layer)

    # Second Convolution Block
    cnn_layer = tf.keras.layers.Conv2D(128, kernel_size=(3, 3))(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(cnn_layer)
    cnn_layer = tf.keras.layers.Dropout(0.3)(cnn_layer)

    # Third Convolution Block
    cnn_layer = tf.keras.layers.Conv2D(64, kernel_size=(3, 3))(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(4, 4))(cnn_layer)
    cnn_layer = tf.keras.layers.Dropout(0.4)(cnn_layer)

    cnn_layer = tf.keras.layers.Flatten()(cnn_layer)

    # Dense Layer
    dense_layer = tf.keras.layers.Dense(128, activation='relu')(cnn_layer)
    dense_layer = tf.keras.layers.Dropout(0.6)(dense_layer)

    # Output Layer
    output = tf.keras.layers.Dense(num_movie, activation='sigmoid', name="movie_scores")(dense_layer)

    # Build and Compile the Model
    model = tf.keras.models.Model(inputs=input_matrix, outputs=output, name="MoviePopularityModel")

    model.compile(
        optimizer="adam",
        loss='mean_squared_error',  # For regression tasks
        metrics=['mae']  # Mean Absolute Error for monitoring
    )

    return model

# Build the model
model = build_cnn_model(num_user, num_movie)

# Train the model
model.fit(
    train_matrix,
    train_matrix,  # For self-supervised learning
    epochs=10,
    batch_size=128
)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.2501 - mae: 0.5001
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.2481 - mae: 0.4979
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - loss: 0.2455 - mae: 0.4926
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - loss: 0.2479 - mae: 0.4899
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - loss: 0.2381 - mae: 0.4646
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - loss: 0.2345 - mae: 0.4737
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step - loss: 0.2212 - mae: 0.4557
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - loss: 0.2180 - mae: 0.4415
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - loss: 0.2031 - mae: 0.3888
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step - loss: 0.2028 - mae: 0.4045


In [66]:
test_matrix_reshaped = test_matrix.reshape((1, num_user, num_movie, 1))
# Predict movie scores
predictions = model.predict(test_matrix_reshaped)

# Aggregate scores across users
movie_scores = predictions.mean(axis=0)

# Get top 50 movie indices
# Top 50 in descending ordere
movie_score = numpy.argsort(movie_scores)[:][::-1]

print("Predicted Movies Ranking(by index):")
for rank, movie_idx in enumerate(movie_score, 1):
    print(f"{rank}: Movie Index {movie_idx}, Score {movie_scores[movie_idx] } ")




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
Predicted Movies Ranking(by index):
1: Movie Index 782, Score 0.6445198655128479 
2: Movie Index 715, Score 0.6281772255897522 
3: Movie Index 96, Score 0.6246629953384399 
4: Movie Index 1266, Score 0.6127764582633972 
5: Movie Index 906, Score 0.6117883324623108 
6: Movie Index 743, Score 0.611548125743866 
7: Movie Index 224, Score 0.6005863547325134 
8: Movie Index 990, Score 0.600132167339325 
9: Movie Index 345, Score 0.5932612419128418 
10: Movie Index 1544, Score 0.5929029583930969 
11: Movie Index 128, Score 0.5922727584838867 
12: Movie Index 1524, Score 0.5913358926773071 
13: Movie Index 507, Score 0.5886155366897583 
14: Movie Index 735, Score 0.5885695219039917 
15: Movie Index 298, Score 0.588015615940094 
16: Movie Index 521, Score 0.5875582695007324 
17: Movie Index 1622, Score 0.5865480899810791 
18: Movie Index 578, Score 0.5837605595588684 
19: Movie Index 43, Score 0.5830947160720825 
20: Movie Index 821, Score 0.5826420187950

In [7]:
# loss, mae = model.evaluate(train_matrix, test_matrix)  # For self-supervised learning, use test_matrix as both inputs and targets
# print(f"Test Loss: {loss}")
# print(f"Test MAE: {mae}")


In [67]:
def calculate_caching_hit_rate(predicted_scores, test_matrix, cache_size_array):

    """
    Calculate the caching hit rate.

    :param predicted_scores: Array of predicted scores for all movies (shape: num_movies).
    :param test_data: Test dataset containing actual movie requests (e.g., movieIDs).
    :param cache_size: Number of movies to cache (top-k based on predicted scores).
    :return: Hit rate as a float.
    """

    predicted_scores = predicted_scores.flatten()


    num_user, num_movie = test_matrix.shape

    for cache_size in cache_size_array:

      # Get top-k movies based on predicted scores
      top_k_movies = numpy.argsort(predicted_scores)[-cache_size:][::-1]

      #Extract actual requested movie IDs from test_matrix
      requested_movie_ids = set()
      hits = 0

      for user_id in range(num_user):

          requested_movies = numpy.where(test_matrix[user_id] > 0)[0]
          requested_movie_ids.update(requested_movies)

      # Calculate hit rate
      for movie_id in requested_movie_ids:
          if movie_id in top_k_movies:
              hits += 1

      total_unique_movies = len(requested_movie_ids)
      print(f"Total unique movies: {total_unique_movies}, total hit: {hits}")

      hit_rate = hits / cache_size
      print(f"Hit Rate for cache size {cache_size}: {hit_rate:.2%}")
      print("")

    return hit_rate

# Example usage
cache_size_array = [50, 100, 150, 200, 250, 300]  # the number of cached top N movies
hit_rate = calculate_caching_hit_rate(predictions, test_matrix, cache_size_array)

Total unique movies: 189, total hit: 6
Hit Rate for cache size 50: 12.00%

Total unique movies: 189, total hit: 11
Hit Rate for cache size 100: 11.00%

Total unique movies: 189, total hit: 17
Hit Rate for cache size 150: 11.33%

Total unique movies: 189, total hit: 26
Hit Rate for cache size 200: 13.00%

Total unique movies: 189, total hit: 33
Hit Rate for cache size 250: 13.20%

Total unique movies: 189, total hit: 41
Hit Rate for cache size 300: 13.67%

